In [21]:
import tensorflow as tf
import keras
import matplotlib
import numpy as np

In [22]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import utils as np_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [24]:
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [25]:
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [26]:
seed = 5
np.random.seed(seed)

In [27]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [29]:
results = []
for index_train, index_test in kfold.split(X_train, np.zeros(shape=(y_train.shape[0], 1))):
    print('Train Index: ', index_train, 'Test Index: ', index_test)
    
    classifier = Sequential()
    
    classifier.add(InputLayer(input_shape=(32, 32, 3)))
    
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    
    classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    
    classifier.add(Flatten())
    
    classifier.add(Dense(128, activation='relu'))
    classifier.add(Dropout(0.25))
    classifier.add(Dense(128, activation='relu'))
    classifier.add(Dropout(0.25))
    
    classifier.add(Dense(10, activation='softmax'))
    
    classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Data augmentation generators
    train_generator = ImageDataGenerator(rotation_range=7, 
                                         horizontal_flip=True, 
                                         shear_range=0.2, 
                                         height_shift_range=0.07, 
                                         zoom_range=0.2)
    test_generator = ImageDataGenerator()
    
    train_data = train_generator.flow(X_train[index_train], y_train[index_train], batch_size=200)
    test_data = test_generator.flow(X_train[index_test], y_train[index_test], batch_size=200)
    
    classifier.fit(train_data, epochs=5, verbose=1)
    prediction = classifier.evaluate(test_data, verbose=0)
    results.append(prediction[1])

Train Index:  [    0     1     2 ... 49997 49998 49999] Test Index:  [    3     4     7 ... 49987 49989 49995]


c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5


c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 65s 299ms/step - accuracy: 0.2829 - loss: 1.9677
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 84s 412ms/step - accuracy: 0.4512 - loss: 1.5168
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 81s 396ms/step - accuracy: 0.5048 - loss: 1.3781
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 83s 407ms/step - accuracy: 0.5411 - loss: 1.2874
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 84s 409ms/step - accuracy: 0.5667 - loss: 1.2247
Train Index:  [    0     1     3 ... 49996 49997 49999] Test Index:  [    2     8    11 ... 49974 49992 49998]
Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 89s 395ms/step - accuracy: 0.2735 - loss: 2.0034
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 85s 418ms/step - accuracy: 0.4606 - loss: 1.4920
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 86s 420ms/step - accuracy: 0.5215 - loss: 1.3393
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 81s 393ms/step - accuracy: 0.5541 - loss: 1.2537
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 81s 398ms/step - accuracy: 0.5799 - loss: 1.1845
Train Index:  [    

In [30]:
results

[0.566100001335144,
 0.5015000104904175,
 0.5902000069618225,
 0.5493999719619751,
 0.5016000270843506]

In [31]:
np.array(results).mean()

0.5417600035667419

In [32]:
np.array(results).std()

0.035301357738572375